In [1]:
import pandas as pd
import networkx as nx
import plotly.express as px

In [2]:
df_links = pd.read_excel("scigrid-conference-eu-data-only/links_eu_power_160718.csvdata.xlsx")
df_nodes = pd.read_excel("scigrid-conference-eu-data-only/vertices_eu_power_160718.csvdata.xlsx")

## Wykresy na mapie

In [5]:
df_nodes.drop(['frequency', 'ref', '')
print(df_nodes.head())
print(df_links.head())

   v_id        lon        lat         typ        voltage frequency  \
0     1   9.522576  52.360409  substation  220000;110000        50   
1     2   9.113210  52.543853  substation         220000        50   
2     3   9.389745  52.026313  substation  380000;110000        50   
3     4   9.125266  52.538264  substation  380000;220000        50   
4     5  10.366275  52.284647  substation  380000;220000        50   

                                 name         operator  ref  \
0                       Hannover/West  TenneT;EON_Netz  NaN   
1  Umspannwerk Kraftwerk Robert Frank           TenneT  NaN   
2          UW Grohnde 380 kV / 110 kV           TenneT  NaN   
3                        Landesbergen  TenneT TSO GmbH  NaN   
4                               Wahle           TenneT  NaN   

                                       wkt_srid_4326  
0  SRID=4326;POINT(9.52257601112577 52.3604090734...  
1  SRID=4326;POINT(9.11321011483187 52.5438533448...  
2  SRID=4326;POINT(9.38974515063535